In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math

from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



try:
    import cPickle as pickle  ##################################
   #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
import time  



import plotly.plotly as py


# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



##### for getting geolocation data  and to calculate distance between two geolocations
import requests
import json
import geopy.distance   




In [2]:
chunksize = 100000   # number of rows i read each time
list_user_ids=[]
header_names_user = ['user_id','email_domain','field','new_group_total_pubs','new_group_num_papers_last','new_group_num_citations']



path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file='DROPBOX_duplicates_removed_no_header.csv'     #results_nico2.csv

df_user=pd.DataFrame(columns=header_names_user)  # i create an empty dataframe
dict_user_basic_attr={}
print df_user.shape
for chunk_df in pd.read_csv(path+input_file, na_values=["NAN","-1","null"],low_memory=False,header=None,  names=header_names_user,usecols=[4,5,12,13,14,15], chunksize=chunksize ):
    lista=list(chunk_df.user_id.unique())
    print chunk_df.shape, len(lista)
    list_user_ids += lista
    print "num. user_ids (and unique)", len(list_user_ids),len(list(set(list_user_ids)) )
    
    df_user=pd.concat([df_user, chunk_df])  # i add new chunks of data every time, but also drop new duplicates
    df_user=df_user.drop_duplicates(subset=["user_id"]) 
    print "building df:",df_user.shape, len(df_user.user_id.unique())                   
    

print "done!"
    

(0, 6)
(100000, 6) 1053
num. user_ids (and unique) 1053 1053
building df: (1053, 6) 1053
(100000, 6) 2886
num. user_ids (and unique) 3939 3797
building df: (3797, 6) 3797
(100000, 6) 3612
num. user_ids (and unique) 7551 7052
building df: (7052, 6) 7052
(100000, 6) 3184
num. user_ids (and unique) 10735 9750
building df: (9750, 6) 9750
(100000, 6) 3203
num. user_ids (and unique) 13938 12366
building df: (12366, 6) 12366
(100000, 6) 4005
num. user_ids (and unique) 17943 15583
building df: (15583, 6) 15583
(100000, 6) 5115
num. user_ids (and unique) 23058 19632
building df: (19632, 6) 19632
(100000, 6) 5408
num. user_ids (and unique) 28466 23808
building df: (23808, 6) 23808
(100000, 6) 3603
num. user_ids (and unique) 32069 26559
building df: (26559, 6) 26559
(100000, 6) 5600
num. user_ids (and unique) 37669 30690
building df: (30690, 6) 30690
(100000, 6) 3607
num. user_ids (and unique) 41276 33206
building df: (33206, 6) 33206
(100000, 6) 5896
num. user_ids (and unique) 47172 37418
buildi

In [3]:

# filename_zip_code_density="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Rural_Urban_zip_codes/Zipcode-ZCTA-Population-Density.csv"
# df_zipcode_density=pd.read_csv(filename_zip_code_density, sep=',')#,usecols=["Zip-ZCTA","2010_Population","Land-Sq-Mi","Density_Per_Sq_Mile"])    #, header=None)
# df_zipcode_density = df_zipcode_density.rename(columns={'Zip-ZCTA': 'zip'})     
# print df_zipcode_density.shape



# filename_zip_rural_urban="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Rural_Urban_zip_codes/t1101_ziprural_simplified.csv" #(31743, 4)
# df_zipcode_rural_urban=pd.read_csv(filename_zip_rural_urban, sep=',',usecols=["zip","cname_abbr","zpop","ru2003"])    #, header=None)
# df_zipcode_rural_urban = df_zipcode_rural_urban.rename(columns={'cname_abbr': 'county_name'})     
# print df_zipcode_rural_urban.shape



filename_zip_rural_urban="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Rural_Urban_zip_codes/2006_Complete_Excel_RUCA_file_simplified.xls"   #(41901, 3)
df_zipcode_rural_urban=pd.read_excel(open(filename_zip_rural_urban))
#pd.read_csv(filename_zip_rural_urban, sep=',',usecols=["zip","RUCA","State_Name"])    #, header=None)
print df_zipcode_rural_urban.shape




filename_rucc_codes="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Rural_Urban_zip_codes/RUCC_codes_simplified.csv"
df_rucc_codes=pd.read_csv(filename_rucc_codes, sep='\t',usecols=["Rural_Urban_Continuum_Code","Description"])    #, header=None)
#df_rucc_codes = df_rucc_codes.rename(columns={'Rural_Urban_Continuum_Code': 'ru2003'})                                                                 
print df_rucc_codes.shape




df_zip_merged_rucc = pd.merge(df_zipcode_rural_urban, df_rucc_codes, how='left',on='Rural_Urban_Continuum_Code')   
print df_zip_merged_rucc.shape
df_zip_merged_rucc.head()


name="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Rural_Urban_zip_codes/merged_zip_RUCC.csv"
df_zip_merged_rucc.to_csv(name, sep=',')
print "written:", name




filename_univ_new="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/University_rankings_and_names/merged_domains_new_clean.csv"
df_univ_names_new=pd.read_csv(filename_univ_new, sep=',',usecols=["email_domain","simplified_domain","University_Name","Cleaned_university_name","Country"])# [email_domain,University_Name,Country] #, header=None)

print df_univ_names_new.shape       



######### transform the data from unicode to strings
for c in df_univ_names_new.columns:#["email_domain","simplified_domain","University_Name","Cleaned_university_name","Country"]:
    df_univ_names_new[c] =df_univ_names_new[c].astype(str)#.str.split(',')
    print c, "done"
    
#df_univ_names_new.head()





df_merged = pd.merge(df_user, df_univ_names_new, how='left',on='email_domain')   
df_merged.rename(columns={'new_group_total_pubs': 'group_total_publ', 'new_group_num_papers_last': 'group_num_papers_last','new_group_num_citations': 'group_num_citations'}, inplace=True)

print df_merged.shape
df_merged.head(10)


(41901, 3)
(12, 2)
(41901, 4)
written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Rural_Urban_zip_codes/merged_zip_RUCC.csv
(7464, 5)
email_domain done
simplified_domain done
University_Name done
Cleaned_university_name done
Country done
(438299, 10)


,user_id,email_domain,field,group_total_publ,group_num_papers_last,group_num_citations,simplified_domain,University_Name,Cleaned_university_name,Country
0,76604.0,unc.edu,humanities,NaN,NaN,NaN,unc.edu,university of north carolina at chapel hill,university of north carolina at chapel hill,United States
1,130678.0,columbia.edu,science,1.0-2.0,0.0-1.0,1.0-30.0,columbia.edu,columbia university,columbia university,United States
2,178708.0,cornell.edu,business econ finances government,NaN,NaN,NaN,cornell.edu,cornell university,cornell university,United States
3,1148983.0,cs.cmu.edu,humanities,NaN,NaN,NaN,cmu.edu,carnegie mellon university,carnegie mellon university,United States
4,278109.0,cs.cmu.edu,humanities,NaN,NaN,NaN,cmu.edu,carnegie mellon university,carnegie mellon university,United States
5,258758.0,iastate.edu,science,22.0-56.0,5.0-18.0,1183.0-146545.0,iastate.edu,iowa state university,iowa state university,United States
6,256616.0,mail.xjtu.edu.cn,NaN,NaN,NaN,NaN,xjtu.edu.cn,xi'an jiaotong university,xi'an jiaotong university,China
7,272328.0,uark.edu,business econ finances government,8.0-22.0,5.0-18.0,229.0-1183.0,uark.edu,university of arkansas,university of arkansas,United States
8,284032.0,walton.uark.edu,NaN,NaN,NaN,NaN,uark.edu,university of arkansas,university of arkansas,United States
9,287646.0,ncsu.edu,science,NaN,NaN,NaN,ncsu.edu,north carolina state university,north carolina state university,United States


In [12]:
print df_merged.shape
df_merged.head(10)

(438299, 10)


,user_id,email_domain,field,group_total_publ,group_num_papers_last,group_num_citations,simplified_domain,University_Name,Cleaned_university_name,Country
0,76604.0,unc.edu,humanities,NaN,NaN,NaN,unc.edu,university of north carolina at chapel hill,university of north carolina at chapel hill,United States
1,130678.0,columbia.edu,science,1.0-2.0,0.0-1.0,1.0-30.0,columbia.edu,columbia university,columbia university,United States
2,178708.0,cornell.edu,business econ finances government,NaN,NaN,NaN,cornell.edu,cornell university,cornell university,United States
3,1148983.0,cs.cmu.edu,humanities,NaN,NaN,NaN,cmu.edu,carnegie mellon university,carnegie mellon university,United States
4,278109.0,cs.cmu.edu,humanities,NaN,NaN,NaN,cmu.edu,carnegie mellon university,carnegie mellon university,United States
5,258758.0,iastate.edu,science,22.0-56.0,5.0-18.0,1183.0-146545.0,iastate.edu,iowa state university,iowa state university,United States
6,256616.0,mail.xjtu.edu.cn,NaN,NaN,NaN,NaN,xjtu.edu.cn,xi'an jiaotong university,xi'an jiaotong university,China
7,272328.0,uark.edu,business econ finances government,8.0-22.0,5.0-18.0,229.0-1183.0,uark.edu,university of arkansas,university of arkansas,United States
8,284032.0,walton.uark.edu,NaN,NaN,NaN,NaN,uark.edu,university of arkansas,university of arkansas,United States
9,287646.0,ncsu.edu,science,NaN,NaN,NaN,ncsu.edu,north carolina state university,north carolina state university,United States


In [13]:
#####  i add university ranking


path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/University_rankings_and_names/"
# input_file='list_top_100universities.tsv'
input_file = "world_ranking2_up_to_1000.csv"
df_univ_ranking=pd.read_csv(path+input_file, sep='\t',na_values=["NAN","-1","null"], usecols=["world_ranking","national_ranking","University"]) # set header=0 if i wanna pass it my own list of header names
df_univ_ranking.rename(columns={'University': 'Cleaned_university_name'}, inplace=True)

print df_univ_ranking.shape  



df_complete = pd.merge(df_merged, df_univ_ranking, how='left',on='Cleaned_university_name') 
print "merged between master file and university ranking."
print df_complete.shape







df_complete.drop('University_Name', axis=1, inplace=True)
df_complete.drop('simplified_domain', axis=1, inplace=True)
df_complete.rename(columns={'Cleaned_university_name': 'University_name'}, inplace=True)



(1000, 3)
merged between master file and university ranking.
(438300, 12)


In [17]:
df_complete.shape

(438300, 11)

In [16]:

######### i add career stage from bin of number of publications    
def get_career_stage (row):
     
    career_stage=np.nan
    max_num_publ=np.nan
             
    
    threshold_experienced=12.    # (see lab notebook: that is the median number of publications in the WOS+LinkedIn population, weighted by how many people are in each macro-field)    
    
       
    try:          
        max_num_publ=float(row.group_total_publ.split("-")[1])        
        career_stage= "jr"        
    except : pass  ## if NAN

     
    
    try:        
        if float(max_num_publ) >= threshold_experienced:
            career_stage= "sr"            
    except ValueError: pass ## if NAN      
    
    
   
    return career_stage
   
    
        
        


%time df_complete['career_stage'] = df_complete.apply (lambda row: get_career_stage(row),axis=1)
print "done"



CPU times: user 11 s, sys: 28 ms, total: 11 s
Wall time: 11 s
done


In [18]:
path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file='new_DROPBOX.csv'


new_name=path+input_file.replace(".csv","")+"_basic_user_attr.csv"
df_complete.to_csv(new_name, sep=';')
print "written:", new_name,"\n\n\n"
df_complete.shape

written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/new_DROPBOX_basic_user_attr.csv 





(438300, 11)

In [11]:
df_complete.shape

(438300, 1)

In [19]:
# filename="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/new_DROPBOX_basic_user_attr.csv"
# df_complete=pd.read_csv(filename, sep=';')
# df_complete.drop('Unnamed: 0', axis=1, inplace=True)
# print df_complete.shape       


dict_var_dict_bins_categories={}
dict_var_dict_bins_categories["group_total_publ"]={}
dict_var_dict_bins_categories["group_num_papers_last"]={}
dict_var_dict_bins_categories["group_num_citations"]={}



dict_var_dict_bins_categories["group_total_publ"]["1.0-2.0"]=1
dict_var_dict_bins_categories["group_total_publ"]["2.0-8.0"]=2
dict_var_dict_bins_categories["group_total_publ"]["8.0-22.0"]=3
dict_var_dict_bins_categories["group_total_publ"]["22.0-56.0"]=4
dict_var_dict_bins_categories["group_total_publ"]["56.0-1350.0"]=5



dict_var_dict_bins_categories["group_num_papers_last"]["0.0-1.0"]=1
dict_var_dict_bins_categories["group_num_papers_last"]["1.0-5.0"]=2
dict_var_dict_bins_categories["group_num_papers_last"]["5.0-18.0"]=3
dict_var_dict_bins_categories["group_num_papers_last"]["18.0-1216.0"]=4



dict_var_dict_bins_categories["group_num_citations"]["0.0-1.0"]=1
dict_var_dict_bins_categories["group_num_citations"]["1.0-30.0"]=2
dict_var_dict_bins_categories["group_num_citations"]["30.0-229.0"]=3
dict_var_dict_bins_categories["group_num_citations"]["229.0-1183.0"]=4
dict_var_dict_bins_categories["group_num_citations"]["1183.0-146545.0"]=5



# group_total_publ
# 1.0-2.0        84560
# 2.0-8.0        42471
# 8.0-22.0       51958
# 22.0-56.0      66337
# 56.0-1350.0    70420me

# group_num_papers_last
# 0.0-1.0        102479
# 1.0-5.0         51978
# 5.0-18.0        74631
# 18.0-1216.0     86658


# group_num_citations
# 0.0-1.0            72323
# 1.0-30.0           48710
# 30.0-229.0         57632
# 229.0-1183.0       64114
# 1183.0-146545.0    72967

#########################

def get_category_from_bins(row, variable_string,dict_var_dict_bins_categories):
    
    category=np.nan            
    
    value=row[variable_string]
    
    try:           
        category= dict_var_dict_bins_categories[variable_string][value]  
#         print row, "\n",variable_string, dict_var_dict_bins_categories[variable_string], "  value:",value, "category:", category
#         raw_input()
    except KeyError: pass  ## if NANs
      
    
    
    return category
        
##################
#   'group_total_publ' 
#   'group_num_papers_last'
#   'group_num_citations'
           
df_complete['category_total_publ'] = df_complete.apply(lambda row: get_category_from_bins(row, 'group_total_publ',dict_var_dict_bins_categories),axis=1)
df_complete['category_total_last_auth'] = df_complete.apply(lambda row: get_category_from_bins(row, 'group_num_papers_last',dict_var_dict_bins_categories),axis=1)
df_complete['category_total_cit'] = df_complete.apply(lambda row: get_category_from_bins(row, 'group_num_citations',dict_var_dict_bins_categories),axis=1)


print "done"


path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file='new_DROPBOX.csv'


new_name=path+input_file.replace(".csv","")+"_basic_user_attr.csv"
df_complete.to_csv(new_name, sep=';')
print "written:", new_name,"\n\n\n"
df_complete.shape

done
written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/new_DROPBOX_basic_user_attr.csv 





(438300, 14)

In [20]:
df_complete


,user_id,email_domain,field,group_total_publ,group_num_papers_last,group_num_citations,University_name,Country,world_ranking,national_ranking,career_stage,category_total_publ,category_total_last_auth,category_total_cit
0,76604.0,unc.edu,humanities,NaN,NaN,NaN,university of north carolina at chapel hill,United States,40.0,25.0,NaN,NaN,NaN,NaN
1,130678.0,columbia.edu,science,1.0-2.0,0.0-1.0,1.0-30.0,columbia university,United States,6.0,4.0,jr,1.0,1.0,2.0
2,178708.0,cornell.edu,business econ finances government,NaN,NaN,NaN,cornell university,United States,10.0,8.0,NaN,NaN,NaN,NaN
3,1148983.0,cs.cmu.edu,humanities,NaN,NaN,NaN,carnegie mellon university,United States,61.0,39.0,NaN,NaN,NaN,NaN
4,278109.0,cs.cmu.edu,humanities,NaN,NaN,NaN,carnegie mellon university,United States,61.0,39.0,NaN,NaN,NaN,NaN
5,258758.0,iastate.edu,science,22.0-56.0,5.0-18.0,1183.0-146545.0,iowa state university,United States,187.0,81.0,sr,4.0,3.0,5.0
6,256616.0,mail.xjtu.edu.cn,NaN,NaN,NaN,NaN,xi'an jiaotong university,China,389.0,13.0,NaN,NaN,NaN,NaN
7,272328.0,uark.edu,business econ finances government,8.0-22.0,5.0-18.0,229.0-1183.0,university of arkansas,United States,550.0,168.0,sr,3.0,3.0,4.0
8,284032.0,walton.uark.edu,NaN,NaN,NaN,NaN,university of arkansas,United States,550.0,168.0,NaN,NaN,NaN,NaN
9,287646.0,ncsu.edu,science,NaN,NaN,NaN,north carolina state university,United States,232.0,93.0,NaN,NaN,NaN,NaN


In [11]:

# group_total_publ
# 1.0-2.0        84560
# 2.0-8.0        42471
# 8.0-22.0       51958
# 22.0-56.0      66337
# 56.0-1350.0    70420

# group_num_papers_last
# 0.0-1.0        102479
# 1.0-5.0         51978
# 5.0-18.0        74631
# 18.0-1216.0     86658


# group_num_citations
# 0.0-1.0            72323
# 1.0-30.0           48710
# 30.0-229.0         57632
# 229.0-1183.0       64114
# 1183.0-146545.0    72967


In [47]:
############ i add geolocation from a dict of university-geolocation

#######   dict univ. geoloc
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_univ_country_geolocation_YAY_.pickle'
with open(pickle_name, 'rb') as handle:
    dict_univ_country_geolocation = pickle.load(handle)
print "num. univ:",len(dict_univ_country_geolocation)
#####################################################




list_univ_not_found=[]
######### i add career stage from bin of number of publications    
def get_geoloc (row, dict_geoloc,list_univ_not_found):
     
    
    geoloc=np.nan
           
    University_name=row.University_name
    Country= row.Country
    
    try:
        string_univ= University_name.lower() +", "+Country
        
        try:
            geoloc= dict_geoloc[string_univ]
        except KeyError:
    #         print row
    #         print string_univ
    #         raw_input()    
            list_univ_not_found.append(string_univ)
 
        
    except AttributeError:   # people without  info on university        
        pass
    
   
    
   
    return geoloc
   
    
        
        


%time df_complete['geoloc'] = df_complete.apply (lambda row: get_geoloc(row, dict_univ_country_geolocation, list_univ_not_found),axis=1)
print "done"







path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file='new_DROPBOX.csv'


new_name=path+input_file.replace(".csv","")+"_basic_user_attr.csv"
df_complete.to_csv(new_name, sep=';', encoding='utf-8')
print "written:", new_name,"\n\n\n"
df_complete.shape


num. univ: 3444
CPU times: user 17.5 s, sys: 148 ms, total: 17.7 s
Wall time: 17.7 s
done
written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/new_DROPBOX_basic_user_attr.csv 





(438466, 15)

In [48]:
df_complete

,user_id,email_domain,field,group_total_publ,group_num_papers_last,group_num_citations,University_name,Country,world_ranking,national_ranking,career_stage,category_total_publ,category_total_last_auth,category_total_cit,geoloc
0,76604.0,unc.edu,humanities,NaN,NaN,NaN,university of north carolina at chapel hill,United States,40,25.0,NaN,NaN,NaN,NaN,"(35.9049122, -79.0469134)"
1,130678.0,columbia.edu,science,1.0-2.0,0.0-1.0,1.0-30.0,columbia university,United States,6,4.0,jr,1.0,1.0,2.0,"(40.8075355, -73.9625727) 10027"
2,178708.0,cornell.edu,business econ finances government,NaN,NaN,NaN,cornell university,United States,10,8.0,NaN,NaN,NaN,NaN,"(42.4534492, -76.4735027) 14850"
3,1148983.0,cs.cmu.edu,humanities,NaN,NaN,NaN,carnegie mellon university,United States,61,39.0,NaN,NaN,NaN,NaN,"(40.4428081, -79.94301279999999) 15213"
4,278109.0,cs.cmu.edu,humanities,NaN,NaN,NaN,carnegie mellon university,United States,61,39.0,NaN,NaN,NaN,NaN,"(40.4428081, -79.94301279999999) 15213"
5,258758.0,iastate.edu,science,22.0-56.0,5.0-18.0,1183.0-146545.0,iowa state university,United States,187,81.0,sr,4.0,3.0,5.0,"(42.0266187, -93.6464654) 50011"
6,256616.0,mail.xjtu.edu.cn,NaN,NaN,NaN,NaN,xi'an jiaotong university,China,389,13.0,NaN,NaN,NaN,NaN,"(34.250575, 108.98371)"
7,272328.0,uark.edu,business econ finances government,8.0-22.0,5.0-18.0,229.0-1183.0,university of arkansas,United States,550,168.0,sr,3.0,3.0,4.0,"(36.0678324, -94.17365509999999) 72701"
8,284032.0,walton.uark.edu,NaN,NaN,NaN,NaN,university of arkansas,United States,550,168.0,NaN,NaN,NaN,NaN,"(36.0678324, -94.17365509999999) 72701"
9,287646.0,ncsu.edu,science,NaN,NaN,NaN,north carolina state university,United States,232,93.0,NaN,NaN,NaN,NaN,"(35.7846633, -78.6820946) 27695"


In [49]:
list_univ_not_found

['die forschungsuniversitat in der helmholtz-gemeinschaft, Germany',
 'universia holding, Spain',
 'fondazione luigi maria, Italy',
 'secretary of iapg as cr v.v.i., Czech Republic',
 'secretary of iapg as cr v.v.i., Czech Republic',
 'fondazione luigi maria, Italy',
 'secretary of iapg as cr v.v.i., Czech Republic',
 'die forschungsuniversitat in der helmholtz-gemeinschaft, Germany',
 'secretary of iapg as cr v.v.i., Czech Republic',
 'fondazione luigi maria, Italy',
 'fondazione luigi maria, Italy']

In [14]:
dict_univ_country_geolocation

{'universite jean moulin lyon 3, France': u'(45.748149, 4.860552999999999) 69008',
 'vincennes university, United States': u'(38.68731, -87.52177879999999) 47591',
 'universidad pontificia comillas, Spain': u'(40.4279716, -3.7152798) 28008',
 'university of leuven-ku, Belgium': u'(50.87795449999999, 4.7002953) 3000',
 'hellenic open university, Greece': u'(38.206531, 21.767111) 263 35',
 'stenden university, Netherlands': u'(53.2112274, 5.7973797) 8917 DD',
 'univerzitetni klinicni center ljubljana, Slovenia': u'(46.0536331, 14.5213538) 1000',
 'notre dame de namur university, United States': u'(37.517299, -122.28487) 94002',
 'dalarna university, Sweden': u'(60.6137678, 15.6536047) 791 31',
 'mount allison university, Canada': u'(45.8983184, -64.3731004) E4L 1H3',
 'rocky mountain university of health professions, United States': u'(40.21185639999999, -111.6560271) 84606',
 'mount vernon nazarene university, United States': u'(40.3736901, -82.4729775) 43050',
 'universidad de las palm

In [30]:
df_complete.head(205)

,user_id,email_domain,field,group_total_publ,group_num_papers_last,group_num_citations,University_name,Country,world_ranking,national_ranking,career_stage,category_total_publ,category_total_last_auth,category_total_cit,geoloc
0,76604.0,unc.edu,humanities,NaN,NaN,NaN,university of north carolina at chapel hill,United States,40,25.0,NaN,NaN,NaN,NaN,"(35.9049122, -79.0469134)"
1,130678.0,columbia.edu,science,1.0-2.0,0.0-1.0,1.0-30.0,columbia university,United States,6,4.0,jr,1.0,1.0,2.0,"(40.8075355, -73.9625727) 10027"
2,178708.0,cornell.edu,business econ finances government,NaN,NaN,NaN,cornell university,United States,10,8.0,NaN,NaN,NaN,NaN,"(42.4534492, -76.4735027) 14850"
3,1148983.0,cs.cmu.edu,humanities,NaN,NaN,NaN,carnegie mellon university,United States,61,39.0,NaN,NaN,NaN,NaN,"(40.4428081, -79.94301279999999) 15213"
4,278109.0,cs.cmu.edu,humanities,NaN,NaN,NaN,carnegie mellon university,United States,61,39.0,NaN,NaN,NaN,NaN,"(40.4428081, -79.94301279999999) 15213"
5,258758.0,iastate.edu,science,22.0-56.0,5.0-18.0,1183.0-146545.0,iowa state university,United States,187,81.0,sr,4.0,3.0,5.0,"(42.0266187, -93.6464654) 50011"
6,256616.0,mail.xjtu.edu.cn,NaN,NaN,NaN,NaN,xi'an jiaotong university,China,389,13.0,NaN,NaN,NaN,NaN,"(34.250575, 108.98371)"
7,272328.0,uark.edu,business econ finances government,8.0-22.0,5.0-18.0,229.0-1183.0,university of arkansas,United States,550,168.0,sr,3.0,3.0,4.0,"(36.0678324, -94.17365509999999) 72701"
8,284032.0,walton.uark.edu,NaN,NaN,NaN,NaN,university of arkansas,United States,550,168.0,NaN,NaN,NaN,NaN,"(36.0678324, -94.17365509999999) 72701"
9,287646.0,ncsu.edu,science,NaN,NaN,NaN,north carolina state university,United States,232,93.0,NaN,NaN,NaN,NaN,"(35.7846633, -78.6820946) 27695"


In [27]:
set(list_univ_not_found)

{'case haiphong medical university, Vietnam',
 'department of chemistry and life sciences, Denmark',
 'die forschungsuniversitat in der helmholtz-gemeinschaft, Germany',
 'flinders university, Australia',
 'fondazione luigi maria, Italy',
 'hacettepe universitesi rektorlugu sihhiye, France',
 'institute of advanced biomedical engineering and science, Japan',
 'jimma university external relations and communications, Ethiopia',
 'kvi-center for advanced radiation technology, Netherlands',
 'school corporation bunkyo university school, Japan',
 'secretary of iapg as cr v.v.i., Czech Republic',
 'universia holding, Spain',
 'universidad esan, Peru',
 'universitatea babescience and technology-bolyai, Romania',
 'universitatea tehnica gheorghe asachi din iasi, Romania',
 'universite jean monnet, France',
 'university hospital medical information network center, Japan',
 'university hospital medical information network, Japan',
 'university magna graecia of catanzaro, Italy',
 'university of 

In [38]:
url_string="http://maps.googleapis.com/maps/api/geocode/json?address="  # this way i access the google api that gives me directly the location of any address or city etc




dict_address_coord={}
for address1 in set(list_univ_not_found):  
    data1 = requests.get(url_string+address1 )  # i make the request for a given site
    jdict1 = json.loads(data1.text)        # i conveninently parse the info using the fact that it is in JSON (it is like a big dictionary of dictionaries)
    try:
        lat1 = jdict1['results'][0]['geometry']['location']['lat']
        lng1 = jdict1['results'][0]['geometry']['location']['lng']
        
        #zip_code = jdict['results'][0]['address_components'][-1]['long_name']  
        
        coords1 = (lat1, lng1)        
        dict_address_coord[address1]=coords1
        print address1, coords1
#         print jdict1
#         raw_input()
    except:
        coords1 = ''
        print address1,"not found!"

    
    
   


# list_addresses=["Bijuesca, Spain","Zaragoza, Spain","Berdejo, Spain","Madrid, Spain","Northwestern University","Yale University, New Haven","Universidad de Zaragoza","Princeton University","Uppsala University","Imperial College","greenwich uk","architectural association school of architecture","delhi, india"]
# dict_address_coord={}
# for address1 in list_addresses:  
#     data1 = requests.get(url_string+address1 )  # i make the request for a given site
#     jdict1 = json.loads(data1.text)        # i conveninently parse the info using the fact that it is in JSON (it is like a big dictionary of dictionaries)
#     try:
#         lat1 = jdict1['results'][0]['geometry']['location']['lat']
#         lng1 = jdict1['results'][0]['geometry']['location']['lng']
#         coords1 = (lat1, lng1)        
#         dict_address_coord[address1]=coords1
#         print address1, coords1
#     except:
#         coords1 = ''
#         print address1,"not found!"

    
    
    
# for address1 in list_addresses:  
#     for address2 in list_addresses:
#         if address != address2:
            
#             coord1=dict_address_coord[address1]
#             coord2=dict_address_coord[address2]
            
#             print address1,"   ---   ", address2
#             print coord1,"   ---   ", coord2
            
#             try:
#                 print geopy.distance.vincenty(coord1, coord2).km, "km", geopy.distance.vincenty(coord1, coord2).miles, "miles,     "
            
#                 print 
#                 print
                
                
#             except: pass
            
            
     


        
        ###### i find by hand the missing geolocations for some university names, and add them to the geoloc. dict:
        
# hacettepe universitesi rektorlugu sihhiye, France not found!   ANKARA, NOT FRANCE      ---------------------  39.931347, 32.865065
# university of curacao, CuraÃŒÂ¤ao not found!  ---------------  12.157344, -68.960586

# die forschungsuniversitat in der helmholtz-gemeinschaft, Germany not found!
# university magna graecia of catanzaro, Italy (38.8686921, 16.5787739)

# institute of advanced biomedical engineering and science, Japan not found!   ----------      35.709806, 139.592113

# flinders university, Australia (-35.0244952, 138.5713281)

# university hospital medical information network, Japan (34.6855436, 135.1710275)

# university hospital medical information network center, Japan (34.6855436, 135.1710275)

# fondazione luigi maria, Italy not found!

# university research institute corporation, Japan (38.9140381, 121.6263303)


# secretary of iapg as cr v.v.i., Czech Republic not found!
# universidad esan, Peru not found! ----------  -12.104555, -76.962008

# universitatea babescience and technology-bolyai, Romania not found!   ------------      46.767928, 23.592306


# case haiphong medical university, Vietnam not found! -------------------   20.836810, 106.726119

# university of nizwa, Guyana (22.8917542, 57.5553994)

# university of prishtina, Albania (42.6484076, 21.1671317)

# school corporation bunkyo university school, Japan (35.9053017, 139.7715211)

# kvi-center for advanced radiation technology, Netherlands (53.2498052, 6.5258788)

# universitatea tehnica gheorghe asachi din iasi, Romania (47.1545419, 27.5994057)
# universia holding, Spain not found!

# department of chemistry and life sciences, Denmark not found!     55.680486, 12.572575
# jimma university external relations and communications, Ethiopia not found!   --------      7.684791, 36.856059
# universite jean monnet, France (45.424622, 4.392984999999999)
        
        
  


hacettepe universitesi rektorlugu sihhiye, France not found!
university of curacao, CuraÃŒÂ¤ao not found!
die forschungsuniversitat in der helmholtz-gemeinschaft, Germany not found!
university magna graecia of catanzaro, Italy (38.8686921, 16.5787739)
{u'status': u'OK', u'results': [{u'geometry': {u'location': {u'lat': 38.8686921, u'lng': 16.5787739}, u'viewport': {u'northeast': {u'lat': 38.87004108029149, u'lng': 16.5801228802915}, u'southwest': {u'lat': 38.8673431197085, u'lng': 16.57742491970849}}, u'location_type': u'GEOMETRIC_CENTER'}, u'address_components': [{u'long_name': u'Viale Europa', u'types': [u'route'], u'short_name': u'Viale Europa'}, {u'long_name': u'Catanzaro', u'types': [u'locality', u'political'], u'short_name': u'Catanzaro'}, {u'long_name': u'Catanzaro', u'types': [u'administrative_area_level_3', u'political'], u'short_name': u'Catanzaro'}, {u'long_name': u'Provincia di Catanzaro', u'types': [u'administrative_area_level_2', u'political'], u'short_name': u'CZ'}, {u'l

KeyboardInterrupt: 

In [45]:


dict_univ_country_geolocation['hacettepe universitesi rektorlugu sihhiye, France']='(39.931347, 32.865065)'
dict_univ_country_geolocation['university of curacao, CuraÃŒÂ¤ao']='(12.157344, -68.960586)'
dict_univ_country_geolocation['university magna graecia of catanzaro, Italy']='(38.8686921, 16.5787739)'
dict_univ_country_geolocation['institute of advanced biomedical engineering and science, Japan']='(35.709806, 139.592113)'
dict_univ_country_geolocation['flinders university, Australia']='(-35.0244952, 138.5713281)'
dict_univ_country_geolocation['university hospital medical information network, Japan']='(34.6855436, 135.1710275)'
dict_univ_country_geolocation['university hospital medical information network center, Japan']='(34.6855436, 135.1710275)'
dict_univ_country_geolocation['university research institute corporation, Japan']='(38.9140381, 121.6263303)'
dict_univ_country_geolocation['universidad esan, Peru']='(-12.104555, -76.962008)'
dict_univ_country_geolocation['universitatea babescience and technology-bolyai, Romania']='(46.767928, 23.592306)'
dict_univ_country_geolocation['case haiphong medical university, Vietnam']='(20.836810, 106.726119)'
dict_univ_country_geolocation['university of nizwa, Guyana']='(22.8917542, 57.5553994)'
dict_univ_country_geolocation['university of prishtina, Albania']='(42.6484076, 21.1671317)'
dict_univ_country_geolocation['school corporation bunkyo university school, Japan']='(35.9053017, 139.7715211)'
dict_univ_country_geolocation['kvi-center for advanced radiation technology, Netherlands']='(53.2498052, 6.5258788)'
dict_univ_country_geolocation['universitatea tehnica gheorghe asachi din iasi, Romania']='(47.1545419, 27.5994057)'
dict_univ_country_geolocation['department of chemistry and life sciences, Denmark']='(55.680486, 12.572575)'
dict_univ_country_geolocation['jimma university external relations and communications, Ethiopia']='(7.684791, 36.856059)'
dict_univ_country_geolocation['universite jean monnet, France']='(45.424622, 4.392984999999999)'



pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_univ_country_geolocation_YAY_.pickle'
with open(pickle_name, 'wb') as handle:
    pickle.dump(dict_univ_country_geolocation, handle)
print "written:", pickle_name


written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_univ_country_geolocation_YAY_.pickle


In [46]:
len(dict_univ_country_geolocation)

3444

In [41]:
df_complete[df_complete["University_name"]=='department of chemistry and life sciences']

,user_id,email_domain,field,group_total_publ,group_num_papers_last,group_num_citations,University_name,Country,world_ranking,national_ranking,career_stage,category_total_publ,category_total_last_auth,category_total_cit,geoloc
68402,26641985.0,bio.aau.dk,NaN,NaN,NaN,NaN,department of chemistry and life sciences,Denmark,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77004,245193254.0,bio.aau.dk,NaN,NaN,NaN,NaN,department of chemistry and life sciences,Denmark,NaN,NaN,NaN,NaN,NaN,NaN,NaN
416243,19214849.0,bio.aau.dk,NaN,NaN,NaN,NaN,department of chemistry and life sciences,Denmark,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
type(dict_univ_country_geolocation['universite jean moulin lyon 3, France'])

unicode